In [278]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon, ranksums, mannwhitneyu

from datasets import binclas_datasets, regr_datasets
from config import dataset_map

In [279]:
postfix = ''

In [280]:
splits = {
    'dtc': pd.read_csv('splits_dtc.csv'),
    'dtr': pd.read_csv('splits_dtr.csv'),
    'rfc': pd.read_csv('splits_rfc.csv'),
    'rfr': pd.read_csv('splits_rfr.csv')
}
evaluations = {
    'dtc': pd.read_csv(f'evaluation_dtc{postfix}.csv'),
    'dtr': pd.read_csv(f'evaluation_dtr{postfix}.csv'),
    'rfc': pd.read_csv(f'evaluation_rfc{postfix}.csv'),
    'rfr': pd.read_csv(f'evaluation_rfr{postfix}.csv')
}

scores = {
    'dtc': 'auc',
    'rfc': 'auc',
    'dtr': 'r2',
    'rfr': 'r2'
}

binclas_order = binclas_datasets[['name']]\
    .apply(lambda row: pd.Series({'name': dataset_map.get(row['name'], row['name'])}), axis=1)
regr_order = regr_datasets[['name']]\
    .apply(lambda row: pd.Series({'name': dataset_map.get(row['name'], row['name'])}), axis=1)

In [281]:
splits = {
    key: value.rename(columns={'Unnamed: 0': 'name'})
    for key, value in splits.items()
}

In [282]:
splits['dtc']['name'] = splits['dtc']['name'].apply(lambda name: dataset_map.get(name, name))
splits['dtr']['name'] = splits['dtr']['name'].apply(lambda name: dataset_map.get(name, name))
splits['rfc']['name'] = splits['rfc']['name'].apply(lambda name: dataset_map.get(name, name))
splits['rfr']['name'] = splits['rfr']['name'].apply(lambda name: dataset_map.get(name, name))

In [283]:
splits['dtc']

,name,n_lattice_splits,n_splits,n_lattice_splits_kfold,n_splits_kfold
0,appendicitis,0,3,1,283
1,haberman,0,7,14,687
2,new_thyroid1,0,2,3,202
3,glass0,0,5,18,557
4,shuttle-6_vs_2-3,0,1,0,100
5,bupa,2,17,158,1652
6,cleveland-0_vs_4,0,3,4,279
7,ecoli1,0,3,17,300
8,poker-9_vs_7,0,2,0,223
9,monk-2,0,5,0,474


In [284]:
for key in splits:
    splits[key]['rate'] = splits[key]['n_lattice_splits'] / splits[key]['n_splits']
    splits[key]['rate_kfold'] = splits[key]['n_lattice_splits_kfold'] / splits[key]['n_splits_kfold']

In [285]:
evaluations = {
    key: value[value['mode'].isin(['<', '<='])]
    for key, value in evaluations.items()
}

In [286]:
evaluations = {
    key: value.groupby(['name', 'mode'])\
            .apply(lambda pdf: pdf.sort_values('fold')[scores[key]].values.tolist())\
            .reset_index(drop=False)\
            .rename(columns={0: scores[key]})
    for key, value in evaluations.items()
}

In [287]:
def evaluate(pdf):
    score = 'auc' if 'auc' in pdf.columns else 'r2'
    leq = pdf[pdf['mode'] == '<='].iloc[0]
    l = pdf[pdf['mode'] == '<'].iloc[0]

    p_less = wilcoxon(leq[score], l[score], alternative='less', zero_method='zsplit').pvalue
    p_gr = wilcoxon(leq[score], l[score], alternative='greater', zero_method='zsplit').pvalue

    return pd.Series({
        f'{score}_diff': np.mean(leq[score]) - np.mean(l[score]),
        'p': min(p_less, p_gr)
    })

In [288]:
figures = {
    key: value.groupby('name').apply(evaluate)
    for key, value in evaluations.items()
}

In [289]:
joined = {
    key: pd.merge(value.reset_index(drop=False), splits[key][['name', 'rate', 'rate_kfold']], on=['name'])\
            .set_index('name')
    for key, value in figures.items()
}

In [290]:
joined['dtc'] = pd.merge(binclas_order, joined['dtc'], on=['name']).set_index('name')
joined['rfc'] = pd.merge(binclas_order, joined['rfc'], on=['name']).set_index('name')
joined['dtr'] = pd.merge(regr_order, joined['dtr'], on=['name']).set_index('name')
joined['rfr'] = pd.merge(regr_order, joined['rfr'], on=['name']).set_index('name')

In [291]:
joined['dtc'] = joined['dtc'][['rate', 'rate_kfold', 'auc_diff', 'p']]
joined['rfc'] = joined['rfc'][['rate', 'rate_kfold', 'auc_diff', 'p']]
joined['dtr'] = joined['dtr'][['rate', 'rate_kfold', 'r2_diff', 'p']]
joined['rfr'] = joined['rfr'][['rate', 'rate_kfold', 'r2_diff', 'p']]

In [292]:
joined['dtc'].columns = pd.MultiIndex.from_tuples(('Decision Tree', col) for col in joined['dtc'].columns)
joined['rfc'].columns = pd.MultiIndex.from_tuples(('Random Forest', col) for col in joined['rfc'].columns)
joined['dtr'].columns = pd.MultiIndex.from_tuples(('Decision Tree', col) for col in joined['dtr'].columns)
joined['rfr'].columns = pd.MultiIndex.from_tuples(('Random Forest', col) for col in joined['rfr'].columns)

In [293]:
binclas = pd.merge(joined['dtc'], joined['rfc'], left_index=True, right_index=True).reset_index(drop=False)
regr = pd.merge(joined['dtr'], joined['rfr'], left_index=True, right_index=True).reset_index(drop=False)

In [294]:
binclas.columns = pd.MultiIndex.from_tuples(('Classification', *col) for col in binclas.columns)
regr.columns = pd.MultiIndex.from_tuples(('Regression', *col) for col in regr.columns)

In [295]:
result = pd.concat([binclas, regr], axis=1)

In [296]:
def formatting(row):
    res = pd.Series()
    for label, value in row.items():
        if label.endswith('diff'):
            string = f'{value:.0e}' if abs(value) > 1e-9 else '0'
            if string[0] != '-':
                string = f' {string}'
            res[label] = string
        elif label.startswith('rate'):
            res[label] = f'{value:.2f}'[1:] if value > 0 else '0'
        elif label == 'p':
            res[label] = '$\\ast$' if value < 0.05 else ''
    return res

In [297]:
result[('Classification', 'Decision Tree')]

/tmp/ipykernel_897/4189619260.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')]


,rate,rate_kfold,auc_diff,p
0,0.000000,0.003534,0.000000,5.000000e-01
1,0.000000,0.020378,-0.000335,4.971797e-04
2,0.000000,0.014851,0.000004,4.119086e-01
3,0.000000,0.032316,0.000000,5.000000e-01
4,0.000000,0.000000,0.000000,5.000000e-01
5,0.117647,0.095642,0.000105,7.051001e-03
6,0.000000,0.014337,-0.003139,7.374231e-02
7,0.000000,0.056667,0.000000,5.000000e-01
8,0.000000,0.000000,0.000000,5.000000e-01
9,0.000000,0.000000,0.000000,5.000000e-01


In [298]:
result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(formatting, axis=1)
result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(formatting, axis=1)
result[('Regression', 'Decision Tree')] = result[('Regression', 'Decision Tree')].apply(formatting, axis=1)
result[('Regression', 'Random Forest')] = result[('Regression', 'Random Forest')].apply(formatting, axis=1)

/tmp/ipykernel_897/513803122.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(formatting, axis=1)
/tmp/ipykernel_897/513803122.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(formatting, axis=1)
/tmp/ipykernel_897/513803122.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(formatting, axis=1)
/tmp/ipykernel_897/513803122.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(formatting, axis=1)
/tmp/ipykernel_897/513803122.py:3: PerformanceWarning: indexing past lexsort depth may impact perfor

In [299]:
result

Classification                                            \
                      name Decision Tree                               
                                    rate rate_kfold auc_diff       p   
0             appendicitis             0        .00        0           
1                 haberman             0        .02   -3e-04  $\ast$   
2             new_thyroid1             0        .01    4e-06           
3                   glass0             0        .03        0           
4         shuttle-6_vs_2-3             0          0        0           
5                     bupa           .12        .10    1e-04  $\ast$   
6         cleveland-0_vs_4             0        .01   -3e-03           
7                   ecoli1             0        .06        0           
8             poker-9_vs_7             0          0        0           
9                   monk-2             0          0        0           
10               hepatitis             0        .04   -2e-04           
11    yeast-0-3-5-9_vs_7-8             0        .04   -4e-05           
12            mammographic             0        .00    3e-05  $\ast$   
13                 saheart             0        .03   -8e-06  $\ast$   
14    page-blocks-1-3_vs_4           .25        .12    1e-04  $\ast$   
15            lymphography             0          0   -4e-03           
16                    pima             0        .05   -3e-04  $\ast$   
17               wisconsin             0        .03   -1e-04  $\ast$   
18             abalone9_18             0          0        0           
19  winequality-red-3_vs_5             0          0        0           

                                                         Regression  \
   Random Forest                                               name   
            rate rate_kfold auc_diff       p                          
0            .02        .03   -4e-06                       diabetes   
1            .04        .05    6e-06                         o-ring   
2            .07        .09    8e-05  $\ast$        stock-portfolio   
3            .05        .05   -2e-05                        wsn-ale   
4            .14        .13        0                   daily-demand   
5            .25        .24    2e-03  $\ast$             slump_test   
6            .09        .11   -8e-04  $\ast$                  servo   
7            .12        .12   -7e-05  $\ast$    yacht_hydrodynamics   
8            .14        .16   -2e-03  $\ast$               autoMPG6   
9            .00        .00        0             excitation_current   
10           .03        .04   -3e-04  $\ast$  real_estate_valuation   
11           .09        .09   -2e-04  $\ast$                wankara   
12           .02        .04    6e-07  $\ast$                plastic   
13           .07        .08    6e-05  $\ast$                  laser   
14           .06        .06   -4e-06          qsar-aquatic-toxicity   
15           .04        .04   -1e-03  $\ast$               baseball   
16           .16        .19    4e-06           maternal_health_risk   
17           .09        .12    4e-05                cpu_performance   
18           .08        .08   -1e-06                        airfoil   
19           .07        .06   -4e-04  $\ast$           medical_cost   

                                                                              \
   Decision Tree                            Random Forest                      
            rate rate_kfold r2_diff       p          rate rate_kfold r2_diff   
0              0        .02  -7e-05                     0        .00   7e-04   
1              0        .03  -4e-03  $\ast$           .13        .10  -2e-02   
2            .05        .03  -1e-04                   .02        .03   2e-05   
3              0        .12  -1e-03                   .12        .11  -1e-03   
4              0        .01  -3e-04  $\ast$           .03        .03  -7e-04   
5            .10        .09   5e-04  $\ast$           .18        .16  -4e-04   
6

In [300]:
result[('Classification', 'name', '')] = result[('Classification', 'name', '')].apply(lambda x: x.replace('_', '-'))
result[('Regression', 'name', '')] = result[('Regression', 'name', '')].apply(lambda x: x.replace('_', '-'))

In [301]:
result

Classification                                            \
                      name Decision Tree                               
                                    rate rate_kfold auc_diff       p   
0             appendicitis             0        .00        0           
1                 haberman             0        .02   -3e-04  $\ast$   
2             new-thyroid1             0        .01    4e-06           
3                   glass0             0        .03        0           
4         shuttle-6-vs-2-3             0          0        0           
5                     bupa           .12        .10    1e-04  $\ast$   
6         cleveland-0-vs-4             0        .01   -3e-03           
7                   ecoli1             0        .06        0           
8             poker-9-vs-7             0          0        0           
9                   monk-2             0          0        0           
10               hepatitis             0        .04   -2e-04           
11    yeast-0-3-5-9-vs-7-8             0        .04   -4e-05           
12            mammographic             0        .00    3e-05  $\ast$   
13                 saheart             0        .03   -8e-06  $\ast$   
14    page-blocks-1-3-vs-4           .25        .12    1e-04  $\ast$   
15            lymphography             0          0   -4e-03           
16                    pima             0        .05   -3e-04  $\ast$   
17               wisconsin             0        .03   -1e-04  $\ast$   
18             abalone9-18             0          0        0           
19  winequality-red-3-vs-5             0          0        0           

                                                         Regression  \
   Random Forest                                               name   
            rate rate_kfold auc_diff       p                          
0            .02        .03   -4e-06                       diabetes   
1            .04        .05    6e-06                         o-ring   
2            .07        .09    8e-05  $\ast$        stock-portfolio   
3            .05        .05   -2e-05                        wsn-ale   
4            .14        .13        0                   daily-demand   
5            .25        .24    2e-03  $\ast$             slump-test   
6            .09        .11   -8e-04  $\ast$                  servo   
7            .12        .12   -7e-05  $\ast$    yacht-hydrodynamics   
8            .14        .16   -2e-03  $\ast$               autoMPG6   
9            .00        .00        0             excitation-current   
10           .03        .04   -3e-04  $\ast$  real-estate-valuation   
11           .09        .09   -2e-04  $\ast$                wankara   
12           .02        .04    6e-07  $\ast$                plastic   
13           .07        .08    6e-05  $\ast$                  laser   
14           .06        .06   -4e-06          qsar-aquatic-toxicity   
15           .04        .04   -1e-03  $\ast$               baseball   
16           .16        .19    4e-06           maternal-health-risk   
17           .09        .12    4e-05                cpu-performance   
18           .08        .08   -1e-06                        airfoil   
19           .07        .06   -4e-04  $\ast$           medical-cost   

                                                                              \
   Decision Tree                            Random Forest                      
            rate rate_kfold r2_diff       p          rate rate_kfold r2_diff   
0              0        .02  -7e-05                     0        .00   7e-04   
1              0        .03  -4e-03  $\ast$           .13        .10  -2e-02   
2            .05        .03  -1e-04                   .02        .03   2e-05   
3              0        .12  -1e-03                   .12        .11  -1e-03   
4              0        .01  -3e-04  $\ast$           .03        .03  -7e-04   
5            .10        .09   5e-04  $\ast$           .18        .16  -4e-04   
6

In [302]:
def rename_columns(col):
    if col == 'rate':
        return '$\\rho$'
    elif col == 'rate_kfold':
        return '$\\rho_{k}$'
    elif col == 'auc_diff':
        return 'auc$_{d}$'
    elif col == 'r2_diff':
        return 'r$^2_{d}$'
    elif col == 'p':
        return 'p$_{\\neq}$'
    else:
        return col

In [303]:
result.columns = pd.MultiIndex.from_tuples([(col[0], col[1], rename_columns(col[2]))
                                            for col in result.columns])

In [304]:
result

Classification                                                \
                      name Decision Tree                                   
                                  $\rho$ $\rho_{k}$ auc$_{d}$ p$_{\neq}$   
0             appendicitis             0        .00         0              
1                 haberman             0        .02    -3e-04     $\ast$   
2             new-thyroid1             0        .01     4e-06              
3                   glass0             0        .03         0              
4         shuttle-6-vs-2-3             0          0         0              
5                     bupa           .12        .10     1e-04     $\ast$   
6         cleveland-0-vs-4             0        .01    -3e-03              
7                   ecoli1             0        .06         0              
8             poker-9-vs-7             0          0         0              
9                   monk-2             0          0         0              
10               hepatitis             0        .04    -2e-04              
11    yeast-0-3-5-9-vs-7-8             0        .04    -4e-05              
12            mammographic             0        .00     3e-05     $\ast$   
13                 saheart             0        .03    -8e-06     $\ast$   
14    page-blocks-1-3-vs-4           .25        .12     1e-04     $\ast$   
15            lymphography             0          0    -4e-03              
16                    pima             0        .05    -3e-04     $\ast$   
17               wisconsin             0        .03    -1e-04     $\ast$   
18             abalone9-18             0          0         0              
19  winequality-red-3-vs-5             0          0         0              

                                                             Regression  \
   Random Forest                                                   name   
          $\rho$ $\rho_{k}$ auc$_{d}$ p$_{\neq}$                          
0            .02        .03    -4e-06                          diabetes   
1            .04        .05     6e-06                            o-ring   
2            .07        .09     8e-05     $\ast$        stock-portfolio   
3            .05        .05    -2e-05                           wsn-ale   
4            .14        .13         0                      daily-demand   
5            .25        .24     2e-03     $\ast$             slump-test   
6            .09        .11    -8e-04     $\ast$                  servo   
7            .12        .12    -7e-05     $\ast$    yacht-hydrodynamics   
8            .14        .16    -2e-03     $\ast$               autoMPG6   
9            .00        .00         0                excitation-current   
10           .03        .04    -3e-04     $\ast$  real-estate-valuation   
11           .09        .09    -2e-04     $\ast$                wankara   
12           .02        .04     6e-07     $\ast$                plastic   
13           .07        .08     6e-05     $\ast$                  laser   
14           .06        .06    -4e-06             qsar-aquatic-toxicity   
15           .04        .04    -1e-03     $\ast$               baseball   
16           .16        .19     4e-06              maternal-health-risk   
17           .09        .12     4e-05                   cpu-performance   
18           .08        .08    -1e-06                           airfoil   
19           .07        .06    -4e-04     $\ast$           medical-cost   

                                                                           \
   Decision Tree                                 Random Forest              
          $\rho$ $\rho_{k}$ r$^2_{d}$ p$_{\neq}$        $\rho$ $\rho_{k}$   
0              0        .02    -7e-05                        0        .00   
1              0        .03    -4e-03     $\ast$           .13        .10   
2            .05        .03    -1e-04                      .02        .03   
3              0        .12    -1e-03                      .12        .11  

In [305]:
latex = result.to_latex(
    index=False,
    multicolumn_format='c'
)

In [306]:
tabular_string = latex[len('\\begin{tabular}{'): len('\\begin{tabular}{') + len(result.columns)]
print(tabular_string)
tabular_string_new = 'lrrrcrrrclrrrcrrrc'
updated = '@{\hspace{4pt}}'.join(tabular_string_new)
latex = latex.replace(tabular_string, updated).replace('name', 'dataset')

llllllllllllllllll


In [307]:
print(latex)

\begin{tabular}{l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}c@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}c@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}c@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}c}
\toprule
\multicolumn{9}{c}{Classification} & \multicolumn{9}{c}{Regression} \\
dataset & \multicolumn{4}{c}{Decision Tree} & \multicolumn{4}{c}{Random Forest} & dataset & \multicolumn{4}{c}{Decision Tree} & \multicolumn{4}{c}{Random Forest} \\
 & $\rho$ & $\rho_{k}$ & auc$_{d}$ & p$_{\neq}$ & $\rho$ & $\rho_{k}$ & auc$_{d}$ & p$_{\neq}$ &  & $\rho$ & $\rho_{k}$ & r$^2_{d}$ & p$_{\neq}$ & $\rho$ & $\rho_{k}$ & r$^2_{d}$ & p$_{\neq}$ \\
\midrule
appendicitis & 0 & .00 &  0 &  & .02 & .03 & -4e-06 &  & diabetes & 0 & .02 & -7e-05 &  & 0 & .00 &  7e-04 & $\ast$ \\
haberman & 0 & .02 & -3e-04 & $\ast$ & .04 & .05 &  6e-06 &  & o-ring & 0 & .03 & -4e-03 & $\ast$ & .13 & .10 & -2e-02 & $\ast$ \\
new-thyroi

In [308]:
with open('tab_presence.tex', 'wt') as file:
    file.write(latex)